# 2. Understanding LLM Input Data

In [1]:
from importlib.metadata import version


print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.8.0
tiktoken version: 0.11.0


- This notebook provides a brief overview of the data preparation and sampling procedures to get input data "ready" for an LLM
- Understanding what the input data looks like is a great first step towards understanding how LLMs work

<img src="./figures/01.png" width="1000px">

<br>
<br>
<br>
<br>

# 2.1 Tokenizing text

- In this section, we tokenize text, which means breaking text into smaller units, such as individual words and punctuation characters

<img src="figures/02.png" width="800px">

- Load raw text we want to work with
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) is a public domain short story

In [2]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- The goal is to tokenize and embed this text for an LLM
- Let's develop a simple tokenizer based on some simple sample text that we can then later apply to the text above

<img src="figures/03.png" width="690px">

- The following regular expression will split on whitespaces and punctuation

In [3]:
import re

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
print(preprocessed[:38])

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '--', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '--', 'so', ' ', 'it', ' ', 'was', ' ', 'no', ' ']


In [4]:
print("Number of tokens:", len(preprocessed))

Number of tokens: 9235


In [5]:
len(set(preprocessed))

1133

In [6]:
sorted(set(preprocessed))

['',
 '\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '--',
 '.',
 ':',
 ';',
 '?',
 'A',
 'Ah',
 'Among',
 'And',
 'Are',
 'Arrt',
 'As',
 'At',
 'Be',
 'Begin',
 'Burlington',
 'But',
 'By',
 'Carlo',
 'Chicago',
 'Claude',
 'Come',
 'Croft',
 'Destroyed',
 'Devonshire',
 'Don',
 'Dubarry',
 'Emperors',
 'Florence',
 'For',
 'Gallery',
 'Gideon',
 'Gisburn',
 'Gisburns',
 'Grafton',
 'Greek',
 'Grindle',
 'Grindles',
 'HAD',
 'Had',
 'Hang',
 'Has',
 'He',
 'Her',
 'Hermia',
 'His',
 'How',
 'I',
 'If',
 'In',
 'It',
 'Jack',
 'Jove',
 'Just',
 'Lord',
 'Made',
 'Miss',
 'Money',
 'Monte',
 'Moon-dancers',
 'Mr',
 'Mrs',
 'My',
 'Never',
 'No',
 'Now',
 'Nutley',
 'Of',
 'Oh',
 'On',
 'Once',
 'Only',
 'Or',
 'Perhaps',
 'Poor',
 'Professional',
 'Renaissance',
 'Rickham',
 'Riviera',
 'Rome',
 'Russian',
 'Sevres',
 'She',
 'Stroud',
 'Strouds',
 'Suddenly',
 'That',
 'The',
 'Then',
 'There',
 'They',
 'This',
 'Those',
 'Though',
 'Thwing',
 'Thwings',
 'To',
 'Usually',
 'Veneti

<br>
<br>
<br>
<br>

# 2.2 Converting tokens into token IDs

- Next, we convert the text tokens into token IDs that we can process via embedding layers later
- For this we first need to build a vocabulary

<img src="figures/04.png" width="900px">

- The vocabulary contains the unique words in the input text

In [7]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1133


In [8]:
for i in enumerate(all_words):
    print(i)

(0, '')
(1, '\n')
(2, ' ')
(3, '!')
(4, '"')
(5, "'")
(6, '(')
(7, ')')
(8, ',')
(9, '--')
(10, '.')
(11, ':')
(12, ';')
(13, '?')
(14, 'A')
(15, 'Ah')
(16, 'Among')
(17, 'And')
(18, 'Are')
(19, 'Arrt')
(20, 'As')
(21, 'At')
(22, 'Be')
(23, 'Begin')
(24, 'Burlington')
(25, 'But')
(26, 'By')
(27, 'Carlo')
(28, 'Chicago')
(29, 'Claude')
(30, 'Come')
(31, 'Croft')
(32, 'Destroyed')
(33, 'Devonshire')
(34, 'Don')
(35, 'Dubarry')
(36, 'Emperors')
(37, 'Florence')
(38, 'For')
(39, 'Gallery')
(40, 'Gideon')
(41, 'Gisburn')
(42, 'Gisburns')
(43, 'Grafton')
(44, 'Greek')
(45, 'Grindle')
(46, 'Grindles')
(47, 'HAD')
(48, 'Had')
(49, 'Hang')
(50, 'Has')
(51, 'He')
(52, 'Her')
(53, 'Hermia')
(54, 'His')
(55, 'How')
(56, 'I')
(57, 'If')
(58, 'In')
(59, 'It')
(60, 'Jack')
(61, 'Jove')
(62, 'Just')
(63, 'Lord')
(64, 'Made')
(65, 'Miss')
(66, 'Money')
(67, 'Monte')
(68, 'Moon-dancers')
(69, 'Mr')
(70, 'Mrs')
(71, 'My')
(72, 'Never')
(73, 'No')
(74, 'Now')
(75, 'Nutley')
(76, 'Of')
(77, 'Oh')
(78, 'On'

In [9]:
vocab = {token:integer for integer,token in enumerate(all_words)}
vocab

{'': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '--': 9,
 '.': 10,
 ':': 11,
 ';': 12,
 '?': 13,
 'A': 14,
 'Ah': 15,
 'Among': 16,
 'And': 17,
 'Are': 18,
 'Arrt': 19,
 'As': 20,
 'At': 21,
 'Be': 22,
 'Begin': 23,
 'Burlington': 24,
 'But': 25,
 'By': 26,
 'Carlo': 27,
 'Chicago': 28,
 'Claude': 29,
 'Come': 30,
 'Croft': 31,
 'Destroyed': 32,
 'Devonshire': 33,
 'Don': 34,
 'Dubarry': 35,
 'Emperors': 36,
 'Florence': 37,
 'For': 38,
 'Gallery': 39,
 'Gideon': 40,
 'Gisburn': 41,
 'Gisburns': 42,
 'Grafton': 43,
 'Greek': 44,
 'Grindle': 45,
 'Grindles': 46,
 'HAD': 47,
 'Had': 48,
 'Hang': 49,
 'Has': 50,
 'He': 51,
 'Her': 52,
 'Hermia': 53,
 'His': 54,
 'How': 55,
 'I': 56,
 'If': 57,
 'In': 58,
 'It': 59,
 'Jack': 60,
 'Jove': 61,
 'Just': 62,
 'Lord': 63,
 'Made': 64,
 'Miss': 65,
 'Money': 66,
 'Monte': 67,
 'Moon-dancers': 68,
 'Mr': 69,
 'Mrs': 70,
 'My': 71,
 'Never': 72,
 'No': 73,
 'Now': 74,
 'Nutley': 75,
 'Of': 76,
 'Oh': 77,
 'On': 78

- Below, we illustrate the tokenization of a short sample text using a small vocabulary:

<img src="figures/05.png" width="800px">

- Let's now put it all together into a tokenizer class

In [10]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- The `encode` function turns text into token IDs
- The `decode` function turns token IDs back into text

<img src="figures/06.png" width="800px">

- We can use the tokenizer to encode (that is, tokenize) texts into integers
- These integers can then be embedded (later) as input of/for the LLM

In [11]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[4, 59, 5, 853, 991, 605, 536, 749, 8, 1129, 599, 8, 4, 70, 10, 41, 854, 1111, 757, 796, 10]


- We can decode the integers back into text

In [12]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [13]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

<br>
<br>
<br>
<br>

# 2.3 BytePair encoding

- GPT-2 used BytePair encoding (BPE) as its tokenizer
- it allows the model to break down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, enabling it to handle out-of-vocabulary words
- For instance, if GPT-2's vocabulary doesn't have the word "unfamiliarword," it might tokenize it as ["unfam", "iliar", "word"] or some other subword breakdown, depending on its trained BPE merges
- The original BPE tokenizer can be found here: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- In this lecture, we are using the BPE tokenizer from OpenAI's open-source [tiktoken](https://github.com/openai/tiktoken) library, which implements its core algorithms in Rust to improve computational performance
- (Based on an analysis [here](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb), I found that `tiktoken` is approx. 3x faster than the original tokenizer and 6x faster than an equivalent tokenizer in Hugging Face)

In [14]:
# pip install tiktoken

In [15]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.11.0


In [16]:
tokenizer = tiktoken.get_encoding("gpt2")

In [17]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [18]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [19]:
# gpt 2 tokenizer can handle unknown words
tokenizer.encode("qwertyuiopasdfghjklzxcvbnm")

[80, 15448, 774, 9019, 404, 292, 7568, 456, 73, 41582, 89, 25306, 85, 9374, 76]

In [20]:
print(f"{tokenizer.decode([80])} {tokenizer.decode([15448])} {tokenizer.decode([774])} {tokenizer.decode([9019])}")

q wer ty ui


- BPE tokenizers break down unknown words into subwords and individual characters:

<img src="figures/07.png" width="700px">

In [21]:
tokenizer.encode("Akwirw ier", allowed_special={"<|endoftext|>"})

[33901, 86, 343, 86, 220, 959]

<br>
<br>
<br>
<br>

# 2.4 Data sampling with a sliding window

- Above, we took care of the tokenization (converting text into word tokens represented as token ID numbers)
- Now, let's talk about how we create the data loading for LLMs
- We train LLMs to generate one word at a time, so we want to prepare the training data accordingly where the next word in a sequence represents the target to predict

<img src="figures/08.png" width="800px">

- For this, we use a sliding window approach, changing the position by +1:

<img src="figures/09.png" width="900px">

- Note that in practice it's best to set the stride equal to the context length so that we don't have overlaps between the inputs (the targets are still shifted by +1 always)

<img src="figures/10.png" width="800px">

In [22]:
from supplementary import create_dataloader_v1


dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("Input Size: ", inputs.size())
print("\nTargets:\n", targets)
print("Target Size: ", targets.size())

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Input Size:  torch.Size([8, 4])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
Target Size:  torch.Size([8, 4])


<br>
<br>
<br>
<br>

# NEXT: Coding an LLM architecture